In [5]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchmetrics.classification import MulticlassRecall, MulticlassPrecision, MulticlassSpecificity

# Image transformer

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Creates a custom dataset. Initialised with the csv file, image file and transformeded.

class messidorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx,0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

# Get the images and csv data file

annotations_file = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor_data.csv"
img_dir = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor-2/preprocess"
af = pd.read_csv(annotations_file)
num_imgs = len(af)

# Splits the dataset into training and testing sets in 80:20 ratio respectively

train_ratio = 0.8
split_id = int(train_ratio * num_imgs)
train_index = (range(0, split_id))
test_index = (range(split_id, num_imgs))

train_dataset = messidorDataset(annotations_file, img_dir, train_transform)
test_dataset = messidorDataset(annotations_file, img_dir, test_transform)

train_dataset = Subset(train_dataset, train_index)
test_dataset = Subset(test_dataset, test_index)

# Loads the dataset, batchsize and shuffles the dataset

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load MobileNetV2

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V2)

# This replaces the final layer of the model to output 5 classes instead of 1000

model.classifier[-1] = torch.nn.Linear(
    in_features=model.classifier[-1].in_features,
    out_features=5
)

# Freeze all layers except the last two layers.

for param in model.parameters():
    param.requires_grad = False
for param in model.features[-2].parameters():
    param.requires_grad = True
for param in model.features[-1].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

# Uses cuda if available or it uses cpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam([
    {"params": model.features[-2].parameters(), "lr": 1e-4},
    {"params": model.features[-1].parameters(), "lr": 1e-4},
    {"params": model.classifier[-1].parameters(), "lr": 1e-3},
])

# Model training

total_epoch = 10
recall_macro = MulticlassRecall(num_classes=5, average="macro").to(device)
precision_macro = MulticlassPrecision(num_classes=5, average="macro").to(device)
specificity_macro = MulticlassSpecificity(num_classes=5, average="macro").to(device)
for epoch in range(total_epoch):
    model.train()
    recall_macro.reset()
    precision_macro.reset()
    specificity_macro.reset()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)

    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
    print(f'Epoch [{epoch+1}/{total_epoch}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')

model.eval()
recall_macro.reset()
precision_macro.reset()
specificity_macro.reset()
correct = 0
total = 0

# Model testing

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)
    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')



Epoch [1/10], Loss: 1.2360, Accuracy: 50.25%, Recall: 0.20, Precision: 0.17, Specificity: 0.80
Epoch [2/10], Loss: 1.0933, Accuracy: 55.91%, Recall: 0.24, Precision: 0.30, Specificity: 0.83
Epoch [3/10], Loss: 1.0265, Accuracy: 59.07%, Recall: 0.27, Precision: 0.54, Specificity: 0.84
Epoch [4/10], Loss: 0.9770, Accuracy: 59.50%, Recall: 0.30, Precision: 0.43, Specificity: 0.85
Epoch [5/10], Loss: 0.9141, Accuracy: 62.44%, Recall: 0.34, Precision: 0.44, Specificity: 0.86
Epoch [6/10], Loss: 0.8722, Accuracy: 65.66%, Recall: 0.41, Precision: 0.51, Specificity: 0.88
Epoch [7/10], Loss: 0.8705, Accuracy: 63.94%, Recall: 0.41, Precision: 0.66, Specificity: 0.87
Epoch [8/10], Loss: 0.8265, Accuracy: 67.10%, Recall: 0.44, Precision: 0.60, Specificity: 0.88
Epoch [9/10], Loss: 0.7860, Accuracy: 68.67%, Recall: 0.48, Precision: 0.67, Specificity: 0.89
Epoch [10/10], Loss: 0.7913, Accuracy: 68.46%, Recall: 0.49, Precision: 0.68, Specificity: 0.89
Test Accuracy: 76.50%, Recall: 0.43, Precision: 0

In [6]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchmetrics.classification import MulticlassRecall, MulticlassPrecision, MulticlassSpecificity

# Image transformer

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Creates a custom dataset. Initialised with the csv file, image file and transformeded.

class messidorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx,0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

# Get the images and csv data file

annotations_file = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor_data.csv"
img_dir = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor-2/preprocess"
af = pd.read_csv(annotations_file)
num_imgs = len(af)

# Splits the dataset into training and testing sets in 80:20 ratio respectively

train_ratio = 0.8
split_id = int(train_ratio * num_imgs)
train_index = (range(0, split_id))
test_index = (range(split_id, num_imgs))

train_dataset = messidorDataset(annotations_file, img_dir, train_transform)
test_dataset = messidorDataset(annotations_file, img_dir, test_transform)

train_dataset = Subset(train_dataset, train_index)
test_dataset = Subset(test_dataset, test_index)

# Loads the dataset, batchsize and shuffles the dataset

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load ResNet18

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# This replaces the final layer of the model to output 5 classes instead of 1000

model.fc = torch.nn.Linear(
    in_features=model.fc.in_features,
    out_features=5
)

# Freeze all layers except the last two layers.

for param in model.parameters():
    param.requires_grad = False
for param in model.layer3.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Uses cuda if available or it uses cpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {"params": model.layer3.parameters(), "lr": 1e-4},
    {"params": model.layer4.parameters(), "lr": 1e-4},
    {"params": model.fc.parameters(), "lr": 1e-3},
])

# Model training

total_epoch = 10
recall_macro = MulticlassRecall(num_classes=5, average="macro").to(device)
precision_macro = MulticlassPrecision(num_classes=5, average="macro").to(device)
specificity_macro = MulticlassSpecificity(num_classes=5, average="macro").to(device)
for epoch in range(total_epoch):
    model.train()
    recall_macro.reset()
    precision_macro.reset()
    specificity_macro.reset()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)

    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
    print(f'Epoch [{epoch+1}/{total_epoch}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')

model.eval()
recall_macro.reset()
precision_macro.reset()
specificity_macro.reset()
correct = 0
total = 0

# Model testing

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)
    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')



Epoch [1/10], Loss: 1.0845, Accuracy: 55.34%, Recall: 0.31, Precision: 0.36, Specificity: 0.85
Epoch [2/10], Loss: 0.8326, Accuracy: 66.31%, Recall: 0.45, Precision: 0.57, Specificity: 0.89
Epoch [3/10], Loss: 0.7098, Accuracy: 71.68%, Recall: 0.57, Precision: 0.67, Specificity: 0.91
Epoch [4/10], Loss: 0.5501, Accuracy: 78.64%, Recall: 0.69, Precision: 0.78, Specificity: 0.93
Epoch [5/10], Loss: 0.4516, Accuracy: 82.44%, Recall: 0.77, Precision: 0.84, Specificity: 0.94
Epoch [6/10], Loss: 0.3199, Accuracy: 87.96%, Recall: 0.82, Precision: 0.88, Specificity: 0.96
Epoch [7/10], Loss: 0.2896, Accuracy: 89.39%, Recall: 0.87, Precision: 0.91, Specificity: 0.97
Epoch [8/10], Loss: 0.2391, Accuracy: 91.76%, Recall: 0.90, Precision: 0.90, Specificity: 0.97
Epoch [9/10], Loss: 0.1836, Accuracy: 93.62%, Recall: 0.92, Precision: 0.94, Specificity: 0.98
Epoch [10/10], Loss: 0.1581, Accuracy: 94.55%, Recall: 0.94, Precision: 0.95, Specificity: 0.98
Test Accuracy: 72.78%, Recall: 0.35, Precision: 0

In [7]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchmetrics.classification import MulticlassRecall, MulticlassPrecision, MulticlassSpecificity

# Image transformer

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Creates a custom dataset. Initialised with the csv file, image file and transformeded.

class messidorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx,0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

# Get the images and csv data file

annotations_file = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor_data.csv"
img_dir = "/mnt/c/Users/matth/CS408/MESSIDOR-2_from_kaggle/messidor-2/preprocess"
af = pd.read_csv(annotations_file)
num_imgs = len(af)

# Splits the dataset into training and testing sets in 80:20 ratio respectively

train_ratio = 0.8
split_id = int(train_ratio * num_imgs)
train_index = (range(0, split_id))
test_index = (range(split_id, num_imgs))

train_dataset = messidorDataset(annotations_file, img_dir, train_transform)
test_dataset = messidorDataset(annotations_file, img_dir, test_transform)

train_dataset = Subset(train_dataset, train_index)
test_dataset = Subset(test_dataset, test_index)

# Loads the dataset, batchsize and shuffles the dataset

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load ResNet18

model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)

# This replaces the final layer of the model to output 5 classes instead of 1000

model.classifier[-1] = torch.nn.Linear(
    in_features=model.classifier[-1].in_features,
    out_features=5
)

# Freeze all layers except the last two layers.

for param in model.parameters():
    param.requires_grad = False
for param in model.features[-2].parameters():
    param.requires_grad = True
for param in model.features[-1].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

# Uses cuda if available or it uses cpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {"params": model.features[-2].parameters(), "lr": 1e-4},
    {"params": model.features[-1].parameters(), "lr": 1e-4},
    {"params": model.classifier[-1].parameters(), "lr": 1e-3},
])

# Model training

total_epoch = 10
recall_macro = MulticlassRecall(num_classes=5, average="macro").to(device)
precision_macro = MulticlassPrecision(num_classes=5, average="macro").to(device)
specificity_macro = MulticlassSpecificity(num_classes=5, average="macro").to(device)
for epoch in range(total_epoch):
    model.train()
    recall_macro.reset()
    precision_macro.reset()
    specificity_macro.reset()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)

    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
    print(f'Epoch [{epoch+1}/{total_epoch}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')

model.eval()
recall_macro.reset()
precision_macro.reset()
specificity_macro.reset()
correct = 0
total = 0

# Model testing

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        recall_macro.update(predicted, labels)
        precision_macro.update(predicted, labels)
        specificity_macro.update(predicted, labels)
    recall = recall_macro.compute().item()
    precision = precision_macro.compute().item()
    specificity = specificity_macro.compute().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%, Recall: {recall:.2f}, Precision: {precision:.2f}, Specificity: {specificity:.2f}')



Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /home/matth/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth


100.0%


Epoch [1/10], Loss: 1.1598, Accuracy: 52.76%, Recall: 0.23, Precision: 0.22, Specificity: 0.82
Epoch [2/10], Loss: 0.9436, Accuracy: 62.80%, Recall: 0.35, Precision: 0.44, Specificity: 0.87
Epoch [3/10], Loss: 0.8457, Accuracy: 65.52%, Recall: 0.44, Precision: 0.68, Specificity: 0.88
Epoch [4/10], Loss: 0.8354, Accuracy: 65.88%, Recall: 0.48, Precision: 0.60, Specificity: 0.88
Epoch [5/10], Loss: 0.7462, Accuracy: 69.96%, Recall: 0.53, Precision: 0.67, Specificity: 0.90
Epoch [6/10], Loss: 0.7235, Accuracy: 70.68%, Recall: 0.58, Precision: 0.68, Specificity: 0.90
Epoch [7/10], Loss: 0.6589, Accuracy: 72.04%, Recall: 0.64, Precision: 0.70, Specificity: 0.91
Epoch [8/10], Loss: 0.6016, Accuracy: 76.42%, Recall: 0.68, Precision: 0.78, Specificity: 0.92
Epoch [9/10], Loss: 0.5605, Accuracy: 77.85%, Recall: 0.70, Precision: 0.81, Specificity: 0.92
Epoch [10/10], Loss: 0.5173, Accuracy: 79.07%, Recall: 0.71, Precision: 0.77, Specificity: 0.93
Test Accuracy: 74.50%, Recall: 0.42, Precision: 0